## This notebook is useful in obtaining **Historical Candle Data** and storing OHCL data for later use

In [1]:
import requests
import pandas as pd
import defs

In [2]:
# Create a session for making HTTP requests
session = requests.Session()

In [3]:
# Define trading instrument, data count, and granularity
instrument = "EUR_USD"
count = 10
granularity = "H1"

In [4]:
# Construct API URL for fetching historical data and print url
url = f"{defs.OANDA_URL}/instruments/{instrument}/candles"

In [5]:
# Define request parameters
params = dict(
    count=count,
    granularity=granularity,
    price="MBA"  # Mid, Bid, and Ask prices
)

In [6]:
# Make GET request with parameters and authentication headers
response = session.get(url, params=params, headers=defs.SECURE_HEADER)
response.status_code

200

In [7]:
# Ensure the response is valid before processing data
if response.status_code != 200:
    print("Error fetching data:", response.text)

In [8]:
# Convert response to JSON
data = response.json()
data.keys()

dict_keys(['instrument', 'granularity', 'candles'])

In [9]:
# Define price types and OHLC (Open, High, Low, Close) components
prices = ['mid', 'bid', 'ask']
ohlc = ['o', 'h', 'l', 'c']

In [10]:
# Print formatted column
for price in prices:
    for oh in ohlc:
        print(f"{price}_{oh}")

mid_o
mid_h
mid_l
mid_c
bid_o
bid_h
bid_l
bid_c
ask_o
ask_h
ask_l
ask_c


In [11]:
# Initialize list for storing structured historical data
historical_data = []
for candle in data.get('candles', []): 
    if not candle.get('complete', False):
        continue 

    # Create a dictionary for storing candle data
    new_dict = {
        'time': candle['time'], 
        'volume': candle['volume']
    }

    # Extract price data for each price type and OHLC component
    for price in prices:
        for oh in ohlc:
            new_dict[f"{price}_{oh}"] = candle[price][oh]

    historical_data.append(new_dict)

In [12]:
# Convert list of dictionaries to a Pandas DataFrame
candles_df = pd.DataFrame.from_dict(historical_data)

In [13]:
# Save DataFrame as a pickle file for later use
candles_df.to_pickle("EUR_USD_H1.pkl")

In [14]:
# Load the saved pickle file to verify data persistence
test_data = pd.read_pickle("EUR_USD_H1.pkl")
test_data

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2025-02-03T18:00:00.000000000Z,12839,1.02795,1.02880,1.02700,1.02785,1.02787,1.02871,1.02692,1.02777,1.02803,1.02889,1.02707,1.02793
1,2025-02-03T19:00:00.000000000Z,6274,1.02785,1.02972,1.02785,1.02846,1.02778,1.02964,1.02778,1.02838,1.02792,1.02980,1.02792,1.02854
2,2025-02-03T20:00:00.000000000Z,5199,1.02848,1.02995,1.02827,1.02956,1.02841,1.02987,1.02820,1.02948,1.02855,1.03003,1.02834,1.02963
3,2025-02-03T21:00:00.000000000Z,13309,1.02954,1.03502,1.02928,1.03428,1.02946,1.03495,1.02921,1.03418,1.02963,1.03509,1.02935,1.03439
4,2025-02-03T22:00:00.000000000Z,1533,1.03380,1.03482,1.03353,1.03433,1.03336,1.03462,1.03311,1.03415,1.03424,1.03501,1.03384,1.03451
5,2025-02-03T23:00:00.000000000Z,5470,1.03427,1.03521,1.03265,1.03281,1.03408,1.03512,1.03258,1.03273,1.03446,1.03530,1.03272,1.03289
6,2025-02-04T00:00:00.000000000Z,4523,1.03284,1.03312,1.03162,1.03179,1.03275,1.03304,1.03154,1.03171,1.03292,1.03319,1.03170,1.03187
7,2025-02-04T01:00:00.000000000Z,6341,1.03178,1.03228,1.03085,1.03220,1.03171,1.03220,1.03076,1.03212,1.03186,1.03235,1.03093,1.03228
8,2025-02-04T02:00:00.000000000Z,5450,1.03220,1.03240,1.03166,1.03222,1.03213,1.03233,1.03158,1.03213,1.03228,1.03249,1.03174,1.03230
